In [1]:
import mne
from sklearn.decomposition import FastICA, PCA
import warnings
from mne.preprocessing import ICA
warnings.filterwarnings('ignore')

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


In [3]:
#Reading sqd file
# input_fname = '/media/analogicalnexus/2568212B752CDB3B/MEG_Data/test_sess_data/R2501_resting_04.04.18.sqd'
input_fname =  '/media/analogicalnexus/2568212B752CDB3B/MEG_Data/NACS642_MEG_Lab_2tone_data/AssignmentData-FRESHCOPY/raw/R2218_two-tones-raw.fif'
stim_list =range(130,188)
raw=mne.io.read_raw_fif(input_fname)#, mrk=None, elp=None, hsp=None,slope='+', stim=stim_list, stimthresh=0.009, stim_code='channel')
raw.load_data()
# raw.plot()

Opening raw data file /media/analogicalnexus/2568212B752CDB3B/MEG_Data/NACS642_MEG_Lab_2tone_data/AssignmentData-FRESHCOPY/raw/R2218_two-tones-raw.fif...
    Range : 0 ... 269999 =      0.000 ...   269.999 secs
Ready.
Current compensation grade : 0
Reading 0 ... 269999  =      0.000 ...   269.999 secs...


<Raw  |  R2218_two-tones-raw.fif, n_channels x n_times : 193 x 270000 (270.0 sec), ~398.6 MB, data loaded>

In [4]:
#Band pass filter
raw_40 = raw.filter(l_freq=0.1, h_freq=60.0)
# raw_40.plot()

Setting up band-pass filter from 0.1 - 60 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 15.0 Hz
Filter length of 66000 samples (66.000 sec) selected


In [5]:
# # compute ICA
# ica = FastICA(n_components=150)
# S_ = ica.fit_transform(raw_40)  # Get the estimated sources
# A_ = ica.mixing_  # Get estimated mixing matrix

In [8]:
events = mne.find_events(raw_40)
# print(events)

200 events found
Events id: [ 9 17]


In [14]:
event_id = dict(tone1=9, tone2=17)  
tmin = -.2 
tmax = 0.5
baseline = (None,0) 
picks = mne.pick_types(raw_40.info, meg=True) 
epochs = mne.Epochs(raw_40, events, event_id, tmin, tmax, proj=False, picks=picks, baseline=baseline) 

200 matching events found


In [19]:
#ICA parameters
n_components = 156  # if float, select n_components by explained variance of PCA
method = 'fastica'  # for comparison with EEGLAB try "extended-infomax" here
decim = 3  # we need sufficient statistics, not all time points -> saves time

# we will also set state of the random number generator - ICA is a
# non-deterministic algorithm, but we want to have the same decomposition
# and the same order of components each time this tutorial is run
random_state = 23

In [ ]:
ica = ICA(n_components=n_components, method=method, random_state=random_state)
# print(ica)
reject = dict(mag=5e-12, grad=4000e-13)
ica.fit(raw, picks=picks, decim=decim, reject=reject)
print(ica)
ica.plot_components()

Fitting ICA to data using 157 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.


In [ ]:
evoked1 = epochs['tone1'].average()
evoked2 = epochs['tone2'].average()
evoked1.plot()
evoked2.plot()

In [ ]:
evoked1.plot(spatial_colors=True, gfp=True, ylim=dict(mag=[-300,300]))
evoked2.plot(spatial_colors=True, gfp=True, ylim=dict(mag=[-300,300]))
evoked1.plot_topomap(times=[.05, .10, .15],vmin=-300,vmax=300)
evoked2.plot_topomap(times=[.05, .10, .15],vmin=-300,vmax=300)
evoked_dict = dict() 
evoked_dict['low'] = evoked1
evoked_dict['high'] = evoked2
colors=dict(low="Crimson",high="CornFlowerBlue") 
mne.viz.plot_compare_evokeds(evoked_dict, colors=colors,
picks=picks, gfp=True)

In [ ]:

la=[0,1,2,3,39,41,42,43,44,52,58,67,71,80,82,83,84,85,108,130,131,132,133,134,135,136,151]
lp=[4,5,6,7,8,9,34,36,37,38,40,45,46,47,48,49,50,75,76,77,79,87,88,90,127,129,137]
ra=[20,22,23,24,26,59,60,61,62,63,65,89,92,95,99,100,114,115,116,117,118,145,147,148,152,155]
rp=[14,15,16,17,18,19,25,27,28,30,53,54,56,57,66,68,69,70,94,96,97,119,121,122,143,144]
lh=[0,1,2,3,39,41,42,43,44,52,58,67,71,80,82,83,84,85,108,130,131,132,133,134,135,136,151,4,5,6,7,8,9,34,36,37,38,40,45,46,47,48,49,50,75,76,77,79,87,88,90,127,129,137]
rh=[20,22,23,24,26,59,60,61,62,63,65,89,92,95,99,100,114,115,116,117,118,145,147,148,152,155, 14,15,16,17,18,19,25,27,28,30,53,54,56,57,66,68,69,70,94,96,97,119,121,122,143,144]
mne.viz.plot_compare_evokeds(evoked_dict, colors=colors,
picks=lh, gfp=True, ylim=dict(mag=[0,100]))